In [1]:
import glob
import numpy as np
import time
import math
import random
from scipy import linalg as LA
import pandas as pd
from sklearn.decomposition import PCA
import os
import csv

In [3]:
def read_file(file_name):
    data = []
    c = 0
    with open(file_name, "r") as f:
        for line in f:
            c = c + 1
            if c > 6:
                item = line.strip().split(",")
                data.append([int(file_name[45:-4])] + list(map(float, item[:2])))
    return data

In [4]:
J = glob.glob('Geolife Trajectories 1.3/**/', recursive=True)[2:]
K = [J[2*i+1] for i in range(182)]
F = [K[i][30:33] for i in range(182)]
int1 = np.vectorize(int)
folder_numbers = int1(F)
I = glob.glob('Geolife Trajectories 1.3/**/*.plt', recursive=True)

In [5]:
# runtime about 65s
Start_time = time.time()
M = 182
data_indexed = [0] * M
data = [0] * M

for i in range(M):
    
    data_indexed[folder_numbers[i]] = []
    data[folder_numbers[i]] = []

    for file_name in I:
        if file_name[30:33] == F[i]:
            a = np.array(read_file(file_name))
            data[folder_numbers[i]].append(a)
            data_indexed[folder_numbers[i]].append(([folder_numbers[i], int(a[0][0]), a[:,1:]]))
    
print('total time =', time.time() - Start_time)

total time = 88.26316595077515


In [6]:
data_indexed[-1][6]

[181, 20080115121304, array([[ 39.97535   , 116.33355   ],
        [ 39.99108333, 116.33171667],
        [ 39.99155   , 116.3306    ],
        [ 39.99153333, 116.32688333],
        [ 39.99136667, 116.31811667],
        [ 39.99108333, 116.31095   ],
        [ 39.9901    , 116.311     ],
        [ 39.98936667, 116.31118333]])]

In [7]:
data_reduced = [0] * M
data_fol_num = set()
trajectory_names = [0] * M

for i in range(M):
    data_reduced[i] = []
    trajectory_names[i] = []
    for j in range(len(data_indexed[i])):
        if len(data_indexed[i][j][2]) > 0: # and sum(sum(abs(data_indexed[i][j][2][:-1] - data_indexed[i][j][2][1:]))) > 10e-3:
            data_reduced[i].append(data_indexed[i][j])
            trajectory_names[i].append((data_indexed[i][j][0], data_indexed[i][j][1]))
            data_fol_num.add(data_indexed[i][j][0])

data_fol_num = np.sort(list(data_fol_num))

In [8]:
trajectory_names[-1][6]

(181, 20080115121304)

In [10]:
l = 0
index = [] 
for i in range(182):
    if len(data_reduced[i]) < 2:
        l = l + 1
    else:
        index.append(i)
        
l, 182-l

(7, 175)

In [11]:
data_1 = [0] * (M-l)
data_fol_num1 = [0] * (M-l)
trajectory_names_1 = [0] * (M-l)

j = 0 
for i in range(M):
    if len(data_reduced[i]) >= 2:
        data_1[j] = np.array(data_reduced[i])
        trajectory_names_1[j] = trajectory_names[i]
        data_fol_num1[j] = data_reduced[i][0][0]
        j = j + 1

In [12]:
# This is doing nothing as we use 0 not 10 for example
data_final = data_1

for i in range(M-l):
    if len(data_1[i]) > 0: #10:
       # R1 = random.sample(range(len(data_1[i])), 200)
       # R = np.sort(R1)
        data_final[i] = data_1[i]#[R]

data_final = np.array(data_final)

In [14]:
# creating box boundaries for landmark points for whole dataset
A = []
B = []
U = []
V = []
M1 = []
M2 = []

for j in range(len(data_final)):
    a = min([min(data_final[j][i][2][:,0]) for i in range(len(data_final[j]))])
    b = max([max(data_final[j][i][2][:,0]) for i in range(len(data_final[j]))])
    u = min([min(data_final[j][i][2][:,1]) for i in range(len(data_final[j]))])
    v = max([max(data_final[j][i][2][:,1]) for i in range(len(data_final[j]))])
    A.append(a)
    B.append(b)
    U.append(u)
    V.append(v)
    
    m1 = np.mean([np.mean(data_final[j][i][2][:,0]) for i in range(len(data_final[j]))])
    m2 = np.mean([np.mean(data_final[j][i][2][:,1]) for i in range(len(data_final[j]))])
    M1.append(np.mean(m1))
    M2.append(np.mean(m2))


print("x-coordinate minimum is: ", min(A))
print("x-coordinate maximum is: ", max(B)) 
print("x-coordinate mean is: ", np.mean(M1))
print("x-coordinate median is: ", np.median(M1))
print("x-coordinate variance is: ", np.var(M1))
print("x-coordinate standard deviation is: ", np.std(M1), '\n')

print("y-coordinate minimum is: ", min(U)) 
print("y-coordinate maximun is: ", max(V))
print("y-coordinate mean is: ", np.mean(M2))
print("y-coordinate median is: ", np.median(M2))
print("y-coordinate variance is: ", np.var(M2))
print("y-coordinate standard deviation is: ", np.std(M2))

x-coordinate minimum is:  1.044024
x-coordinate maximum is:  400.166666666667
x-coordinate mean is:  39.33880381206802
x-coordinate median is:  39.9562137187774
x-coordinate variance is:  7.35607627483406
x-coordinate standard deviation is:  2.712208744701274 

y-coordinate minimum is:  -179.9695933
y-coordinate maximun is:  179.9969416
y-coordinate mean is:  115.22263199878307
y-coordinate median is:  116.3387035363077
y-coordinate variance is:  60.74033571549907
y-coordinate standard deviation is:  7.7936086452617745


In [22]:
# We have used the following Q as landmarks

Q = np.array([[39.6,114.8],[40,116.34], [39, 114], [40,115], [41,116], [39.5,113.5], 
                [40.5, 115.5], [40.3, 114.4], [38.5, 113.5], [39.3,117], [39.5,115], 
                [39.4, 116], [39,114.6], [40.5, 115.5], [41.2,115], [42, 116], [41.7, 114.8], 
               [38,115], [37, 116], [37.5, 114]])


In [21]:
def dist_signed_point_closed(Q, gamma, sigma): 
    
    p1 = gamma[:-1]
    p2 = gamma[1:]
    L = np.sqrt(((p2-p1)*(p2-p1)).sum(axis =1)) + 10e-6
    
    w = (p1-p2)*(-1,1)/(L * np.ones((2,1))).T
    w[:,[0, 1]] = w[:,[1, 0]]
    
# signed distance to the extended lines of segments
    dist_signed = np.sum(w * (Q.reshape(len(Q),1,2) - p1), axis=2)
    x = abs(dist_signed.copy())
    R = (L**2).reshape(-1,1)
# u = argmin points on the extended lines of segments
    u = p1 + ((((np.sum(((Q.reshape(len(Q),1,2) - p1) * (p2 - p1)),axis=2).reshape(len(Q)
                ,-1,1,1) * (p2-p1).reshape(len(p2-p1),1,2))).reshape(len(Q),len(p1),2))/R)

    G = np.sqrt(np.sum((u-p1)*(u-p1), axis=2))
    H = np.sqrt(np.sum((u-p2)*(u-p2), axis=2))
# d1 = distance to start points
    d1 = np.sqrt(np.sum((Q.reshape(len(Q),1,2)-p1)*(Q.reshape(len(Q),1,2)-p1), axis=2))
# d2 = distance to end points
    d2 = np.sqrt(np.sum((Q.reshape(len(Q),1,2)-p2)*(Q.reshape(len(Q),1,2)-p2), axis=2))
    d = np.where(d1 < d2, d1, d2)
    dist_segment = np.where(abs(G + H - L) < np.ones(len(L)) * (10e-6), dist_signed, d)
    
    J2 = [0] * len(Q)
    for i in range(len(Q)): 
        J2[i] = np.where(abs(G + H - L)[i] > 10e-6)[0]
    J2 = np.array(J2)

    dist_segment_copy = dist_segment.copy()
    dist = abs(dist_segment_copy)


    j = np.argmin(dist, axis =1)

    sign = np.ones(len(Q))
    for k in range(len(Q)): 
        if j[k] in J2[k]:
            if j[k] == 0 and LA.norm(Q[k] - gamma[0]) < LA.norm(Q[k] - gamma[1]):
                
                y = LA.norm(gamma[0]-gamma[1]) - LA.norm(gamma[-1] - gamma[-2])
                if y < 0:
                    x = gamma[0] + 0.1 * LA.norm(gamma[0]-gamma[1])*(gamma[-2]-gamma[-1])/LA.norm(gamma[-2]-gamma[-1])
                    z = gamma[0] + 0.1 * LA.norm(gamma[0]-gamma[1])*(gamma[1]-gamma[0])/LA.norm(gamma[1]-gamma[0])
                    q = 2 * gamma[0] - (x + z)/2
                else: 
                    x = gamma[0] + 0.1 * LA.norm(gamma[-1]-gamma[-2])*(gamma[1]-gamma[0])
                    z = gamma[0] + 0.1 * LA.norm(gamma[-1]-gamma[-2])*(gamma[-2]-gamma[-1])
                    q = 2 * gamma[0] - (x + z)/2
                sign[k] = np.sign((q-gamma[-1]).dot(w[-1] + w[0]))
                
            elif j[k] == len(gamma)-2 and LA.norm(Q[k] - gamma[-1]) < LA.norm(Q[k] - gamma[-2]):
                s = w[-1].dot((Q[k] - gamma[-1])/ LA.norm(Q[k] - gamma[-1]) + 10e-6)
                sign[k] = np.sign(s)
            
            elif LA.norm(Q[k] - gamma[j[k]]) < LA.norm(Q[k] - gamma[j[k]+1]):  
                q = 2 * gamma[j[k]] - (gamma[j[k]-1] + gamma[j[k]+1])/2
                sign[k] = np.sign((q-gamma[j[k]]).dot(w[j[k]-1] + w[j[k]]))
                    
            elif LA.norm(Q[k] - gamma[j[k]+1]) <= LA.norm(Q[k] - gamma[j[k]]):
                q = 2 * gamma[j[k]+1] - (gamma[j[k]] + gamma[j[k]+2])/2
                sign[k] = np.sign((q-gamma[j[k]+1]).dot(w[j[k]] + w[j[k]+1]))

    E = dist_segment[np.arange(len(dist_segment)),j] 
    F = dist[np.arange(len(dist)),j] 
    dist_weighted = sign * (1/sigma) * (E.reshape(-1,1) * np.exp(-(F/sigma)**2).reshape(-1,1)).reshape(1,-1)

    return dist_weighted.reshape(len(Q))

In [22]:
def dist_signed_point_unclosed(Q, gamma, sigma): 
    
    p1 = gamma[:-1]
    p2 = gamma[1:]
    L = np.sqrt(((p2-p1)*(p2-p1)).sum(axis =1)) + 10e-6
    w = (p1-p2)*(-1,1)/(L * np.ones((2,1))).T
    w[:,[0, 1]] = w[:,[1, 0]]
    
# signed distance to the extended lines of segments
    dist_signed = np.sum(w * (Q.reshape(len(Q),1,2) - p1), axis=2)
    x = abs(dist_signed.copy())
    R = (L**2).reshape(-1,1)
# u = argmin points on the extended lines of segments
    u = p1 + ((((np.sum(((Q.reshape(len(Q),1,2) - p1) * (p2 - p1)),axis=2).reshape(len(Q)
                ,-1,1,1) * (p2-p1).reshape(len(p2-p1),1,2))).reshape(len(Q),len(p1),2))/R)

    G = np.sqrt(np.sum((u-p1)*(u-p1), axis=2))
    H = np.sqrt(np.sum((u-p2)*(u-p2), axis=2))
# d1 = distance to start points
    d1 = np.sqrt(np.sum((Q.reshape(len(Q),1,2)-p1)*(Q.reshape(len(Q),1,2)-p1), axis=2))
# d2 = distance to end points
    d2 = np.sqrt(np.sum((Q.reshape(len(Q),1,2)-p2)*(Q.reshape(len(Q),1,2)-p2), axis=2))
    d = np.where(d1 < d2, d1, d2)
    dist_segment = np.where(abs(G + H - L) < np.ones(len(L)) * (10e-6), dist_signed, d)
    
    J2 = [0] * len(Q)
    for i in range(len(Q)): 
        J2[i] = np.where(abs(G + H - L)[i] > 10e-6)[0]
    J2 = np.array(J2)

    dist_segment_copy = dist_segment.copy()
    dist = abs(dist_segment_copy)
    
    dist_from_start_1 = np.sqrt(((Q -p1[0])*(Q -p1[0])).sum(axis =1))
    ds_1 = ((Q -p1[0])*w[0]).sum(axis =1)
    #dist_from_start = np.sign(ds_1) * (abs(ds_1) + np.sqrt(dist_from_start_1**2 - ds_1**2 + 10e-6))
    dist_from_start = ds_1 * np.maximum(abs(ds_1), np.sqrt(dist_from_start_1**2 - ds_1**2 + 10e-6))/ (dist_from_start_1 + 10e-6)

    
    dist_from_end_1 = np.sqrt(((Q -p2[-1])*(Q -p2[-1])).sum(axis =1))
    de_1 = ((Q -p2[-1])* w[-1]).sum(axis =1)
    #dist_from_end = np.sign(de_1) * (abs(de_1) + np.sqrt(dist_from_end_1**2 - de_1**2 + 10e-6))
    dist_from_end = de_1 * np.maximum(abs(de_1), np.sqrt(dist_from_end_1**2 - de_1**2 + 10e-6))/ (dist_from_end_1+ 10e-6)


    dist_segment[:,0] = np.where(abs(dist[:,0]- dist_from_start_1)< 10e-8, dist_from_start, dist_segment[:,0]) 
    dist_segment[:,-1] = np.where(abs(dist[:,-1]- dist_from_end_1)< 10e-8, dist_from_end, dist_segment[:,-1]) 


    j = np.argmin(dist, axis =1)

    sign = np.ones(len(Q))
    for k in range(len(Q)): 
        if j[k] in J2[k]: 
            if j[k] == 0 and LA.norm(Q[k] - gamma[0]) < LA.norm(Q[k] - gamma[1]):
                sign[k] = 1
                
            elif j[k] == len(gamma)-2 and LA.norm(Q[k] - gamma[j[k]+1]) < LA.norm(Q[k] - gamma[j[k]]):
                sign[k] = 1
            
            elif LA.norm(Q[k] - gamma[j[k]]) < LA.norm(Q[k] - gamma[j[k]+1]):  
                q = 2 * gamma[j[k]] - (gamma[j[k]-1] + gamma[j[k]+1])/2
                sign[k] = np.sign((q-gamma[j[k]]).dot(w[j[k]-1] + w[j[k]]))
                    
            elif LA.norm(Q[k] - gamma[j[k]+1]) <= LA.norm(Q[k] - gamma[j[k]]) and j[k]+2 <=len(gamma)-1:
                q = 2 * gamma[j[k]+1] - (gamma[j[k]] + gamma[j[k]+2])/2
                sign[k] = np.sign((q-gamma[j[k]+1]).dot(w[j[k]] + w[j[k]+1]))

    E = dist_segment[np.arange(len(dist_segment)),j] 
    F = dist[np.arange(len(dist)),j] 
    dist_weighted = sign * (1/sigma) * (E.reshape(-1,1) * np.exp(-(F/sigma)**2).reshape(-1,1)).reshape(1,-1)

    return dist_weighted.reshape(len(Q))

In [23]:
def dist_signed_point(Q, gamma, sigma):
    if LA.norm(gamma[0]-gamma[-1]) > 10e-6:
        A = dist_signed_point_unclosed(Q, gamma, sigma)
    else: 
        A = dist_signed_point_closed(Q, gamma, sigma)
    return A

In [91]:
r = M-l
sigma = 0.3
#projected_curves = [0] * r # in R^20

In [93]:
a = 0
b = M-l

In [103]:
Start_time = time.time()

c = 0
projected_curves = []

for i in range(a,b):

    Start_time_1 = time.time()
    
    for j in range(len(data_final[i])):
    
        B = [c, data_final[i][j][0], data_final[i][j][1]]
        E = dist_signed_point(Q,data_final[i][j][2],sigma)
        projected_curves.append(list(np.concatenate((B,E), axis = 0)))
        c += 1
    print(i,'time =', time.time() - Start_time_1)

projected_curves = np.array(projected_curves)

print('total time =', time.time() - Start_time)

0 time = 0.21025824546813965
1 time = 0.10016894340515137
2 time = 0.22803616523742676
3 time = 0.42160892486572266
4 time = 0.48256993293762207
5 time = 0.10966014862060547
6 time = 0.03148484230041504
7 time = 0.08115506172180176
8 time = 0.06966590881347656
9 time = 0.0736241340637207
10 time = 0.7286269664764404
11 time = 0.13499999046325684
12 time = 0.13640880584716797
13 time = 0.21736598014831543
14 time = 0.33016300201416016
15 time = 0.07529902458190918
16 time = 0.06971192359924316
17 time = 0.7496922016143799
18 time = 0.046662330627441406
19 time = 0.055626869201660156
20 time = 0.15541696548461914
21 time = 0.005216121673583984
22 time = 0.36648011207580566
23 time = 0.10653209686279297
24 time = 0.21529293060302734
25 time = 0.7160632610321045
26 time = 0.11850810050964355
27 time = 0.013322114944458008
28 time = 0.10760903358459473
29 time = 0.06875395774841309
30 time = 0.5239360332489014
31 time = 0.0162351131439209
32 time = 0.021596908569335938
33 time = 0.046960830

In [89]:
Start_time = time.time()

projected = [0] * (M-l)

for i in range(11):

    projected[i] = []
    
    for j in range(len(data_final[i])):
        projected[i].append(dist_signed_point(Q,data_final[i][j][2],sigma))
        
    projected[i] = np.array(projected[i])
        
projected = np.array(projected)

print('total time =', time.time() - Start_time)

total time = 2.5710010528564453


In [112]:
path = '1/2-dim.csv'
np.savetxt(path,projected_curves, delimiter=',')

In [113]:
header = ["key_0", "User Name", "Trajectory Name", "x", "y"]

path_1 = r'1/2-dim-headed.csv'

with open(path_1, "w", newline='') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(header) # write the header
    
    # write the actual content line by line
    with open(path, 'r', newline='') as incsv:
        reader = csv.reader(incsv)
        writer.writerows(row for row in reader)

In [ ]:
header = ["User Name", "Trajectory Name", "x_1", "x_2", "x_3", "x_4", "x_5", "x_6", "x_7", "x_8", 
         "x_9", "x_10", "x_11", "x_12", "x_13", "x_14", "x_15", "x_16", "x_17", "x_18", 
          "x_19", "x_20"]

path_1 = r'1/Vis_Vecs-2+20-dim-larger-than-1-trajs-all-in-1-file/Vis_Vecs-2+20-dim-larger-than-1-trajs-all-in-1-file-headed.csv'

with open(path_1, "w", newline='') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(header) # write the header
    
    # write the actual content line by line
    with open(path, 'r', newline='') as incsv:
        reader = csv.reader(incsv)
        writer.writerows(row for row in reader)

In [87]:
projected_curves[-1]

array([ 1.81000000e+02,  2.00712080e+13, -1.43206652e-11, -3.98097972e-02,
       -5.25783807e-30, -3.23709648e-08, -1.61583386e-05, -1.11949420e-38,
       -1.50177284e-04, -9.26529605e-18, -3.54071551e-48,  1.81104934e-03,
       -2.41228173e-09,  2.76368632e-02, -1.52855520e-18, -1.50177284e-04,
       -3.88746919e-15, -8.70175395e-20, -9.09305166e-25,  2.76594044e-26,
        3.59716717e-40,  8.05560290e-55])

In [193]:
newpath = r'Vis_Vectors-20-dimensional-larger-than-1-trajectories-one-by-one/0' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [240]:
for i in range(a,b):
    
    strr = r'Vis_Vectors-20-dimensional-larger-than-1-trajectories-one-by-one/'
    newpath = strr + str(data_fol_num1[i])
    newpath = newpath + '/'
    
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    
    for j in range(len(projected_curves[i])):

        np.savetxt(newpath+str(trajectory_names_1[i][j][1])+'.csv', projected_curves[i][j][2], 
                   delimiter=',')
        

In [241]:
newpath = r'Vis_Vectors-20-dimensional-larger-than-1-trajectories-one-by-one-all/'

for i in range(a,b):    
    for j in range(len(projected_curves[i])):
        np.savetxt(newpath+str(trajectory_names_1[i][j])+'.csv', 
                                   projected_curves[i][j][2], delimiter=',')


# Dimensionality Reduction with PCA

In [88]:
X = projected_curves[:,2:]
pca = PCA(n_components=2)
pca.fit(X)
X_Reduced = pca.transform(X)
X_concatenated = np.concatenate((projected_curves[:,:2], X_Reduced),axis=1)

path = r'1/Vis_Vectors-2+20-to-2+2-dim-larger-than-1-trajs-all-in-1-file/Vis_Vectors-2+20-to-2+2-dim-larger-than-1-trajs-all-in-1-file.csv'
np.savetxt(path, X_concatenated, delimiter=',')


In [100]:
header = ["User Name", "Trajectory Name", "x", "y"]

path_2 = r'1/Vis_Vectors-2+20-to-2+2-dim-larger-than-1-trajs-all-in-1-file/Vis_Vectors-2+20-to-2+2-dim-larger-than-1-trajs-all-in-1-file-headed.csv'
path = r'1/Vis_Vectors-2+20-to-2+2-dim-larger-than-1-trajs-all-in-1-file/Vis_Vectors-2+20-to-2+2-dim-larger-than-1-trajs-all-in-1-file.csv'

with open(path_2, "w", newline='') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(header) # write the header
    
    # write the actual content line by line
    with open(path, 'r', newline='') as incsv:
        reader = csv.reader(incsv)
        writer.writerows(row for row in reader)

## For Clustering

In [264]:
# for clustering

path1 = r'Vis_Vectors-20-to-2-PCA-one-by-one-all/'

for i in range(M-l):
    X = list(projected_curves[i][:,2])
    pca = PCA(n_components=2)
    pca.fit(X)
    X_Reduced = pca.transform(X)
    
    strr = r'Vis_Vectors-20-to-2-PCA-one-by-one/'
    newpath = strr + str(data_fol_num1[i])
    newpath = newpath + '/'
    
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    
    for j in range(len(X_Reduced)):

        np.savetxt(newpath+str(trajectory_names_1[i][j][1])+'.csv', X_Reduced[j], delimiter=',')
    
        np.savetxt(path1+str(trajectory_names_1[i][j])+'.csv', X_Reduced[j], delimiter=',')
    

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/pca.py:447: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/pca.py:447: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var


In [42]:
for i in range(182-l):
    X = projected_curves[i]
    pca = PCA(n_components=5)
    pca.fit(X)
    X_Reduced = pca.transform(X)
    np.savetxt('20-to-2-PCA-128/'+str(data_fol_num1[i])+'.csv', X_Reduced, delimiter=',')


In [119]:
import json

with open('/Users/hasan/Desktop/1.json') as f:
    data = json.load(f)
#print(data)

In [ ]:
def make_json(csvFilePath, jsonFilePath): 
      
    # create a dictionary 
    data = {} 
      
    # Open a csv reader called DictReader 
    with open(csvFilePath, encoding='utf-8') as csvf: 
        csvReader = csv.DictReader(csvf) 
          
        # Convert each row into a dictionary  
        # and add it to data 
        for rows in csvReader: 
              
            # Assuming a column named 'No' to 
            # be the primary key 
            key = rows['No'] 
            data[key] = rows 
  
    # Open a json writer, and use the json.dumps()  
    # function to dump data 
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
        jsonf.write(json.dumps(data, indent=4)) 
          
# Driver Code 
 
csvFilePath = r'/Users/hasan/Desktop/1.csv'
jsonFilePath = r'/Users/hasan/Desktop/2.json'
  
# Call the make_json function 
make_json(csvFilePath, jsonFilePath)